In [1]:
import xpress as xp

In [5]:
def dummyCallback():
    print("Callback called")

In [ ]:
# What we need to feed this model is the following:
# 1- List of donor-receipient pairs
# 2- List of altruisitic donors
# 3- List of nodes = donor-receipient pairs + altruisitic donors
# 4- Dictionary of edges with key ("node1","node2") and value is the weight of the edge
# 5- Dictionary of cycles with key (edge1,edge2,edge3,...) and value is the weight of the cycle i.e., sum of edges in the cycle up to and including length k
pairs = ["P1", "P2", "P3"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 10, 
         ("NDD1", "P2"): 8,
         ("P1", "P3"): 8, 
         ("P2", "P3"): 9,
         ("P3", "P1"): 6,
         ("P1", "P2"): 6}
all_cycles = {}






# Define the optimization model
model = xp.problem()



# Decision variables
y = {e: xp.var(vartype=xp.binary, name=f"y_{e}") for e in edges}  # Edge selection
z = {c: xp.var(vartype=xp.binary, name=f"z_{c}") for c in all_cycles}  # Cycle selection
f_i = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow in decision variable
f_o = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow out decision variable

model.addVariable(list(y.values()) + list(z.values())+ list(f_i.values()) + list(f_o.values()))



# Define the objective function which is to maximize sum of weights of selected edges and cycles
model.setObjective(xp.Sum(edges[e] * y[e] for e in edges) + xp.Sum(all_cycles[c] * z[c] for c in all_cycles), sense=xp.maximize)




### Constraints:

# 1. Defining f_i and f_o i.e., the incoming and outgoing kidneys to a node v
for v in nodes:
    model.addConstraint(xp.Sum([y[e] for e in edges if e[1] == v]) == f_i[v])
    model.addConstraint(xp.Sum([y[e] for e in edges if e[0] == v]) == f_o[v])

# 2. Ensure that if a node is in an actived cycle, the associated edges that involve it are turned off
for v in pairs:
    model.addConstraint(f_o[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]))
    model.addConstraint(f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= 1)

# 3. Ensure that altruistic donors donate at most one kidney  
for v in altruistic_donors:
    model.addConstraint(f_o[v] <= 1)

# 4. The hard and confusing constraint... This version enumerates all possible edges. I just added for completeness
# i, k = 2, 3
# subsets = [subset for j in range(i, k+1) for subset in itertools.combinations(pairs, j)]
# for S in subsets:
#     for v in S:
#          model.addConstraint(xp.Sum([y[e] for e in edges if (e[1] in S and e[0] not in S)]) >= f_i[v])



# Add the call back function to detect and add violated constraints during LP relaxation
model.addcbpreintsol(dummyCallback, None, 3)

# Solve the model
model.solve()

#
# print(model.getProbStatusString())

# # Get the current solution
# current_sol = ({e: model.getSolution(y[e]) for e in edges}, {c: model.getSolution(z[c]) for c in all_cycles}, {v: model.getSolution(f_i[v]) for v in nodes})

# # Extract and print solution
# print("Objective Value:", model.getObjVal())
# selected_edges = [e for e in edges if model.getSolution(y[e]) > 0.5]
# selected_cycles = [c for c in all_cycles if model.getSolution(z[c]) > 0.5]
# print("Selected Edges:", selected_edges)
# print("Selected Cycles:", selected_cycles)

FICO Xpress v9.4.2, Hyper, solve started 15:56:08, Feb 12, 2025
Heap usage: 394KB (peak 394KB, 165KB system)
Maximizing MILP noname using up to 16 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
        15 rows           14 cols           30 elements        14 entities
Presolved problem has:
         7 rows            9 cols           18 elements         9 entities
Presolve finished in 0 seconds
Heap usage: 425KB (peak 447KB, 165KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 6.00e+00,  1.00e+01] / [ 6.00e+00,  1.00e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 8.9GB
User interrupt (7) triggered.
 *** S

(<SolveStatus.STOPPED: 1>, <SolStatus.NOTFOUND: 0>)